# Compare les balises TCOF selon la source de téléchargement

Imports

In [1]:
import os
import pandas as pd
import re
import requests
import xml.etree.ElementTree as ET
import numpy as np

Fonctions

In [2]:
def get_all_folder(path):
    directory = os.listdir(path)
    folder_list = []
    for folder in directory:
        if os.path.isdir(os.path.join(path, folder)):
            folder_list.append(folder)
    return folder_list

In [3]:
def add_path2corpus(path,corpus):
    new = '{}\{}'.format(path, corpus)
    return new

In [4]:
def flatten_list(liste):
    flattened_list = []
    for sublist in liste:
        for item in sublist:
            flattened_list.append(item)
    return flattened_list

In [5]:
def get_corpus_files(path):
    files = {}
    for file in os.listdir(path):
        if file.endswith('.xml'):
            files['xml']=file
        if file.endswith('.trs'):
            pattern = r'(anonymise_[\S]*.trs)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_trs']=file
            else:
                files['trs']=file
        if file.endswith('.wav'):
            pattern = r'(anonymise_[\S]*.wav)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_wav']=file
            else:
                files['wav']=file
        else:
            continue
    return files

In [6]:
def get_files_df(path):
    folders_genre=[add_path2corpus(path, folder) for folder in get_all_folder(path)] 
    list_corpus_path = []
    for folder in folders_genre:
        corpus_path = [add_path2corpus(folder, corpus) for corpus in get_all_folder(folder)]
        list_corpus_path.append(corpus_path)
    list_corpus_path = flatten_list(list_corpus_path)
    mydict = {}
    for corpus_path in list_corpus_path:
        dict_corpus = get_corpus_files(corpus_path)
        mydict[corpus_path]=dict_corpus
    df = pd.DataFrame.from_dict(mydict, orient='index').rename_axis('corpus').reset_index()
    return df

In [7]:
def add_corpus_names_col(df):
    wav = df['wav'].tolist()
    corpus_names = []
    for file in wav:
        pattern = r'^(.*)\.wav$'
        match = re.search(pattern, file)
        if match:
            corpus = match.group(1)
        else:
            corpus = "NAN"
        corpus_names.append(corpus)
    df['corpus_name'] = corpus_names
    return df

In [8]:
def get_all_tagsname(xml_file):
    # Parse the XML file into an ElementTree object
    try : 
        tree = ET.parse(xml_file)
    # Get the root element of the tree
        root = tree.getroot()
    # Initialize an empty set to store the tag names
        tag_names = set()
    # Iterate over all elements in the tree and add their tag names to the set
        for elem in root.iter():
            tag_names.add(elem.tag)
    # Return the set of tag names
    except: 
        return "Pas de fichier trouvé à cet emplacement"
    return tag_names

In [9]:
def is_same(list1, list2):
    s = set(list1)
    difference = [x for x in list2 if x not in s]
    if difference : 
        return difference
    else: 
        return True

In [10]:
def get_corpus_name(path):
    filename = re.search(r'[^\\\/]*?(?=\.\w+$)', path).group()
    return filename

In [11]:
def get_df_tags(xml_files, source):
    corpus_name = [get_corpus_name(xml_file) for xml_file in xml_files]
    corpus_source = [source for xml_file in range(len(xml_files))]
    tags_ = [get_all_tagsname(xml_file) for xml_file in xml_files]
    df = pd.DataFrame(list(zip(corpus_name, corpus_source, tags_)), columns = ['corpus_name','source','tags'])
    return df

In [23]:
def add_corpus_names_col(df):
    wav = df['xml'].tolist()
    corpus_names = []
    for file in wav:
        try: 
            pattern = r'^(.*)\.xml$'
            match = re.search(pattern, file)
            if match:
                corpus = match.group(1)
        except:
            corpus = 'NAN'
        corpus_names.append(corpus)
    df['corpus_name'] = corpus_names
    return df

In [24]:
def get_files_cnrtl(path):
    #folders_genre=[add_path2corpus(path, folder) for folder in get_all_folder(path)] 
    list_corpus_path = [add_path2corpus(path, corpus) for corpus in get_all_folder(path)]
    mydict = {}
    for corpus_path in list_corpus_path:
        dict_corpus = get_corpus_files(corpus_path)
        mydict[corpus_path]=dict_corpus
    df = pd.DataFrame.from_dict(mydict, orient='index').rename_axis('corpus').reset_index()
    return df

In [25]:
def get_corpus_files(path):
    files = {}
    for file in os.listdir(path):
        if file.endswith('.xml'):
            files['xml']=file
        if file.endswith('.trs'):
            pattern = r'(anonymise_[\S]*.trs)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_trs']=file
            else:
                files['trs']=file
        if file.endswith('.wav'):
            pattern = r'(anonymise_[\S]*.wav)'
            match = re.search(pattern, file)
            if match:
                files['anonymise_wav']=file
            else:
                files['wav']=file
        else:
            continue
    return files

In [26]:
def add_path2corpus(path,corpus):
    new = '{}\{}'.format(path, corpus)
    return new

In [27]:
def get_all_folder(path):
    directory = os.listdir(path)
    folder_list = []
    for folder in directory:
        if os.path.isdir(os.path.join(path, folder)):
            folder_list.append(folder)
    return folder_list

In [36]:
def get_difference(df):
    # prendre la liste des tags + l'intersection de toutes les listes
    lists_tags = df['tags'].tolist()
    results_union = set().union(*lists_tags)
    # trouver les différences entre l'intersection de toutes les listes et la liste des tags d'un corpus donné
    df['is_same'] = df['tags'].apply(lambda x:is_same(x, results_union))
    return results_union, df

## Analyse de TCOF ZIP (depuis ortolang.fr)

In [12]:
#mettre le chemin du dossier téléchargé
path_zip = 'E:/Corpus/tcof/tcof/12/Corpus/Adultes'
#obtenir le dataframe avec le noms des fichiers du dossier dedans
df_zip = get_files_df(path_zip)

In [13]:
df_zip = add_corpus_names_col(df_zip)

In [14]:
df_zip

,corpus,wav,trs,xml,corpus_name
0,E:/Corpus/tcof/tcof/12/Corpus/Adultes\GenreNon...,plaid_haut_07.wav,plaid_haut_07.trs,plaid_haut_07.xml,plaid_haut_07
1,E:/Corpus/tcof/tcof/12/Corpus/Adultes\GenreNon...,sdl_bas_14.wav,sdl_bas_14.trs,sdl_bas_14.xml,sdl_bas_14
2,E:/Corpus/tcof/tcof/12/Corpus/Adultes\GenreNon...,forum_gom_14.wav,forum_gom_14.trs,forum_gom_14.xml,forum_gom_14
3,E:/Corpus/tcof/tcof/12/Corpus/Adultes\GenreNon...,voyage_jus_14.wav,voyage_jus_14.trs,voyage_jus_14.xml,voyage_jus_14
4,E:/Corpus/tcof/tcof/12/Corpus/Adultes\GenreNon...,chev_beu_sd.wav,chev_beu_sd.trs,chev_beu_sd.xml,chev_beu_sd
...,...,...,...,...,...
234,E:/Corpus/tcof/tcof/12/Corpus/Adultes\Conversa...,automobile_gue_08.wav,automobile_gue_08.trs,automobile_gue_08.xml,automobile_gue_08
235,E:/Corpus/tcof/tcof/12/Corpus/Adultes\Conversa...,tourisme_arn_15.wav,tourisme_arn_15.trs,tourisme_arn_15.xml,tourisme_arn_15
236,E:/Corpus/tcof/tcof/12/Corpus/Adultes\Conversa...,photographie_cou_14.wav,photographie_cou_14.trs,photographie_cou_14.xml,photographie_cou_14
237,E:/Corpus/tcof/tcof/12/Corpus/Adultes\Conversa...,voyage_con_15.wav,voyage_con_15.trs,voyage_con_15.xml,voyage_con_15


In [15]:
#créer la liste de tous les chemins de fichier xml du tcof_zip
xml_files_zip = [r'{}\{}.xml'.format(x, y).replace('\\','/') for x,y in list(zip(df_zip['corpus'].tolist(), df_zip['corpus_name'].tolist()))]

In [16]:
#remplir le dataframe 
df_zip = get_df_tags(xml_files_zip, "zip_ortolang")

In [17]:
df_zip

,corpus_name,source,tags
0,plaid_haut_07,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
1,sdl_bas_14,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
2,forum_gom_14,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
3,voyage_jus_14,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
4,chev_beu_sd,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
...,...,...,...
234,automobile_gue_08,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
235,tourisme_arn_15,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
236,photographie_cou_14,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
237,voyage_con_15,zip_ortolang,"{alignement, canal, type, nom_corpus, commenta..."


# TCOF REPOSITORY

In [18]:
#remettre ici le chemin du dossier où les fichiers ont été téléchargés
directory_path = r'E:/Corpus/tcof/TCOF_repo/'
directory_files = os.listdir(directory_path)
xml_files_repo = []
for file in directory_files:
    if file.endswith('.xml'):
        xml_files_repo.append(file)

In [19]:
xml_files_repo = ['{}{}'.format(directory_path, x) for x in xml_files_repo]

In [20]:
df = get_df_tags(xml_files_repo, "repository_ortolang")

In [21]:
df_repo = df.copy()

In [22]:
df_repo

,corpus_name,source,tags
0,these_pit_09,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
1,consultation_ger_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
2,orthophonie2_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
3,orthophonie_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
4,pedi_gra_06,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
...,...,...,...
234,reunionkholle_moi_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
235,reunionpp_aar_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
236,see_reu_mat_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
237,taki_cam_13,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."


# TCOF CNRTL 

Télécharger tous liens des fichiers .xml, .trs et .wav pour chaque corpus

In [28]:
#mettre le chemin du dossier 
path_cnrtl = r'E:\Corpus\TCOF_CNRTL'
#obtenir le dataframe avec le noms des fichiers du dossier dedans
df_cnrtl = get_files_cnrtl(path_cnrtl)


In [29]:
#ajouter les noms de colonnes
df_cnrtl = add_corpus_names_col(df_cnrtl)

In [30]:
xml_files_cnrtl = [r'{}\{}.xml'.format(x, y).replace('\\','/') for x,y in list(zip(df_cnrtl['corpus'].tolist(), df_cnrtl['corpus_name'].tolist()))]

In [31]:
df_cnrtl = get_df_tags(xml_files_cnrtl, "CNRTL")

In [32]:
#transformer les lignes sans fichier
df_cnrtl = df_cnrtl.replace('Pas de fichier trouvé à cet emplacement',np.NaN)

In [33]:
df_cnrtl

,corpus_name,source,tags
0,theorielinguistique_07,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
1,testssncf_qab_12,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
2,theatre_fle_11,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
3,telephone_lam_13,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
4,tel_maz_07,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
...,...,...,...
584,thibaut1_der,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
585,thibault1_lev,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
586,thibault1_cor,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
587,thibaud1_son,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."


# Comparaison des balises

In [34]:
#réunir toutes les dataframes dans un seul
df = df.append(df_zip)
df = df.append(df_cnrtl.loc[df_cnrtl['tags'].notna()])

C:\Users\maeva\AppData\Local\Temp\ipykernel_10392\2599307716.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_zip)
C:\Users\maeva\AppData\Local\Temp\ipykernel_10392\2599307716.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_cnrtl.loc[df_cnrtl['tags'].notna()])


In [35]:
df

,corpus_name,source,tags
0,these_pit_09,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
1,consultation_ger_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
2,orthophonie2_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
3,orthophonie_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
4,pedi_gra_06,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta..."
...,...,...,...
584,thibaut1_der,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
585,thibault1_lev,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
586,thibault1_cor,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."
587,thibaud1_son,CNRTL,"{modalite_recueil, naissance, canal, type, nom..."


**Comparaison de tous les fichiers xml**

In [37]:
results_union, df = get_difference(df)

In [38]:
print('Nb de tags communs :', len(results_union), '\n Liste des tags communs :\n', results_union)

Nb de tags communs : 89 
 Liste des tags communs :
 {'alignement', 'canal', 'type', 'nom_corpus', 'lien_autres_corpus', 'commentaire', 'responsable', 'autorisation', 'situation_enonciation', 'document_annexe', 'particularite', 'relation_locuteur', 'corpusOraux', 'anonymisation', 'travaux', 'general', 'resume', 'debut_timecode_transcription', 'dernier_timecode_transcription', 'typologie', 'note', 'age', 'createur_fiche', 'mots_clefs', 'date', 'condition', 'appartenance', 'status', 'arrondissement', 'modalite_recueil', 'auteur', 'titre', 'statut_francais', 'taille', 'nombre_mots', 'convention_transcription', 'locuteur', 'nombre_locuteurs', 'duree_transcription', 'lieu', 'pays', 'type_corpus', 'format', 'droit_acces', 'support_dialogue', 'ancien_identifiant', 'degre', 'description', 'sexe', 'editeur', 'enregistrement', 'laboratoire', 'duree', 'nom', 'relation', 'numerique', 'profession_actuelle', 'page', 'description_lieu', 'qualite', 'region', 'annee', 'residence', 'nomDossier', 'ville',

In [39]:
df

,corpus_name,source,tags,is_same
0,these_pit_09,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
1,consultation_ger_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
2,orthophonie2_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
3,orthophonie_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
4,pedi_gra_06,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
...,...,...,...,...
584,thibaut1_der,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
585,thibault1_lev,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
586,thibault1_cor,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
587,thibaud1_son,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."


**Comparaison des fichiers repository ortolang entre eux**

In [40]:
results_union_repo, df_repo = get_difference(df_repo)

In [41]:
print('Nb de tags communs :', len(results_union_repo), '\n Liste des tags communs :\n', results_union_repo)

Nb de tags communs : 78 
 Liste des tags communs :
 {'alignement', 'canal', 'type', 'nom_corpus', 'commentaire', 'responsable', 'autorisation', 'document_annexe', 'particularite', 'corpusOraux', 'anonymisation', 'travaux', 'general', 'resume', 'typologie', 'note', 'age', 'mots_clefs', 'date', 'condition', 'appartenance', 'status', 'modalite_recueil', 'auteur', 'titre', 'statut_francais', 'taille', 'nombre_mots', 'locuteur', 'nombre_locuteurs', 'lieu', 'pays', 'type_corpus', 'format', 'droit_acces', 'support_dialogue', 'ancien_identifiant', 'degre', 'description', 'sexe', 'editeur', 'enregistrement', 'laboratoire', 'duree', 'nom', 'relation', 'numerique', 'profession_actuelle', 'page', 'description_lieu', 'qualite', 'region', 'annee', 'residence', 'nomDossier', 'ville', 'nom_fichier', 'profession_anterieure', 'projet_associe', 'mail', 'responsable_corpus', 'temps_parole', 'naissance', 'referencement', 'transcription', 'association', 'formation', 'reference_support', 'support', 'reviseur

**Liste différents tags par fichier**

In [42]:
for index, row in df_repo.iterrows():
    if row['is_same'] == True:
        continue
    else:
        print('Nom du corpus: ', row['corpus_name'],' \t Tags différents:', row['is_same'])

Nom du corpus:  these_pit_09  	 Tags différents: ['reviseur']
Nom du corpus:  consultation_ger_08  	 Tags différents: ['reviseur']
Nom du corpus:  orthophonie2_san_10  	 Tags différents: ['reviseur']
Nom du corpus:  orthophonie_san_10  	 Tags différents: ['reviseur']
Nom du corpus:  apiculteur_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  avoixnue_06  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  decesprofesseur_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  deuilradio_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  enfant_lem_10  	 Tags différents: ['reviseur']
Nom du corpus:  erasmusespagne_sd  	 Tags différents: ['reviseur']
Nom du corpus:  etudiantesalariee_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  footballeurprofessionnel_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  joueurmusique_02  	 Tags différents: ['reviseur']
Nom du corpus:  lecturecroco_sd 

In [43]:
df_repo

,corpus_name,source,tags,is_same
0,these_pit_09,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",[reviseur]
1,consultation_ger_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",[reviseur]
2,orthophonie2_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",[reviseur]
3,orthophonie_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",[reviseur]
4,pedi_gra_06,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",True
...,...,...,...,...
234,reunionkholle_moi_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",[reviseur]
235,reunionpp_aar_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",True
236,see_reu_mat_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",True
237,taki_cam_13,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...",True


**Comparaison des fichiers zip_ortolang entre eux**

In [44]:
results_union_zip, df_zip = get_difference(df_zip)

In [45]:
print('Nb de tags communs :', len(results_union_zip), '\n Liste des tags communs :\n', results_union_zip)

Nb de tags communs : 78 
 Liste des tags communs :
 {'alignement', 'canal', 'type', 'nom_corpus', 'commentaire', 'responsable', 'autorisation', 'document_annexe', 'particularite', 'corpusOraux', 'anonymisation', 'travaux', 'general', 'resume', 'typologie', 'note', 'age', 'mots_clefs', 'date', 'condition', 'appartenance', 'status', 'modalite_recueil', 'auteur', 'titre', 'statut_francais', 'taille', 'nombre_mots', 'locuteur', 'nombre_locuteurs', 'lieu', 'pays', 'type_corpus', 'format', 'droit_acces', 'support_dialogue', 'ancien_identifiant', 'degre', 'description', 'sexe', 'editeur', 'enregistrement', 'laboratoire', 'duree', 'nom', 'relation', 'numerique', 'profession_actuelle', 'page', 'description_lieu', 'qualite', 'region', 'annee', 'residence', 'nomDossier', 'ville', 'nom_fichier', 'profession_anterieure', 'projet_associe', 'mail', 'responsable_corpus', 'temps_parole', 'naissance', 'referencement', 'transcription', 'association', 'formation', 'reference_support', 'support', 'reviseur

**Liste des différences de tags par fichier**

In [46]:
for index, row in df_zip.iterrows():
    if row['is_same'] == True:
        continue
    else:
        print('Nom du corpus: ', row['corpus_name'],' \t Tags différents:', row['is_same'])

Nom du corpus:  sculpture_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  explorationssonores  	 Tags différents: ['transcripteur']
Nom du corpus:  pompe_bli_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  gestapo_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  sncf_dez_11  	 Tags différents: ['reviseur']
Nom du corpus:  paralysie  	 Tags différents: ['transcripteur']
Nom du corpus:  communication_bon_sd  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  guerre_coc_sd  	 Tags différents: ['transcripteur']
Nom du corpus:  tatouagepolynesie_06  	 Tags différents: ['reviseur']
Nom du corpus:  theorielinguistique_07  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  apprendreaulycee  	 Tags différents: ['transcripteur']
Nom du corpus:  recherchecancer_06  	 Tags différents: ['reviseur', 'transcripteur']
Nom du corpus:  boxe_sd  	 Tags différents: ['transcripteur']
Nom du corpus:  ouvrier_pal_10  	 

**Comparaison des fichiers cnrtl entre eux**

In [47]:
results_union_cnrtl_tags, df_cnrtl = get_difference(df_cnrtl.loc[df_cnrtl['tags'].notna()])

In [48]:
print('Nb de tags communs :', len(results_union_cnrtl_tags), '\n Liste des tags communs :\n', results_union_cnrtl_tags)

Nb de tags communs : 70 
 Liste des tags communs :
 {'canal', 'type', 'nom', 'nom_corpus', 'lien_autres_corpus', 'relation', 'commentaire', 'responsable', 'autorisation', 'situation_enonciation', 'document_annexe', 'particularite', 'relation_locuteur', 'corpusOraux', 'anonymisation', 'profession_actuelle', 'general', 'resume', 'description_lieu', 'qualite', 'debut_timecode_transcription', 'region', 'dernier_timecode_transcription', 'residence', 'nomDossier', 'ville', 'nom_fichier', 'profession_anterieure', 'age', 'createur_fiche', 'date', 'mail', 'appartenance', 'responsable_corpus', 'temps_parole', 'arrondissement', 'modalite_recueil', 'naissance', 'tcof_version', 'referencement', 'titre', 'statut_francais', 'transcription', 'taille', 'nombre_mots', 'convention_transcription', 'locuteur', 'formation', 'support', 'nombre_locuteurs', 'reviseur', 'duree_transcription', 'etude', 'pays', 'type_corpus', 'role', 'transcripteur', 'format', 'logiciel_alignement', 'droit_acces', 'support_dialog

In [49]:
for index, row in df_cnrtl.iterrows():
    if row['is_same'] == True:
        continue
    else:
        print('Nom du corpus: ', row['corpus_name'],' \t Tags différents:', row['is_same'])

In [50]:
df_cnrtl

,corpus_name,source,tags,is_same
0,theorielinguistique_07,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
1,testssncf_qab_12,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
2,theatre_fle_11,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
3,telephone_lam_13,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
4,tel_maz_07,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
...,...,...,...,...
584,thibaut1_der,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
585,thibault1_lev,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
586,thibault1_cor,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True
587,thibaud1_son,CNRTL,"{modalite_recueil, naissance, canal, type, nom...",True


**DF complet (toutes les sources)**

In [51]:
df

,corpus_name,source,tags,is_same
0,these_pit_09,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
1,consultation_ger_08,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
2,orthophonie2_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
3,orthophonie_san_10,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
4,pedi_gra_06,repository_ortolang,"{alignement, canal, type, nom_corpus, commenta...","[lien_autres_corpus, situation_enonciation, re..."
...,...,...,...,...
584,thibaut1_der,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
585,thibault1_lev,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
586,thibault1_cor,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."
587,thibaud1_son,CNRTL,"{modalite_recueil, naissance, canal, type, nom...","[alignement, travaux, typologie, note, mots_cl..."


In [52]:
df.to_csv(r'E:\Corpus\tags_TCOF.csv', index=False,)  